<a href="https://colab.research.google.com/github/huggingface/deep-rl-class/blob/main/notebooks/bonus-unit1/bonus_unit1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="In Colab öffnen"/></a>

# Bonuseinheit 1: Huggy, der Hund 🐶, soll einen Stock apportieren.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit2/thumbnail.png" alt="Bonus Unit 1Thumbnail">

In diesem Heft festigen wir das in der ersten Einheit Gelernte, indem wir **Huggy, dem Hund, beibringen, das Stöckchen zu holen und dann direkt in deinem Browser damit zu spielen**

⬇️ Hier ist ein Beispiel für das, was **am Ende der Einheit erreicht wird** ⬇️ (▶ starten, um zu sehen)

In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.mp4" type="video/mp4"></video>

### Die Umwelt 🎮

- Huggy the Dog, eine von [Thomas Simonini](https://twitter.com/ThomasSimonini) geschaffene Umgebung, basierend auf [Puppo The Corgi](https://blog.unity.com/technology/puppo-the-corgi-cuteness-overload-with-the-unity-ml-agents-toolkit)

### Die verwendete Bibliothek 📚

- [MLAgents](https://github.com/Unity-Technologies/ml-agents)

Wir versuchen ständig, unsere Anleitungen zu verbessern. **Wenn Sie also Probleme in diesem Notizbuch** finden, öffnen Sie bitte [ein Problem im Github Repo](https://github.com/huggingface/deep-rl-class/issues).

## Ziele dieses Notizbuchs 🏆

Am Ende des Notizbuchs werden Sie:

- Den **Zustandsraum, den Aktionsraum und die Belohnungsfunktion, die für das Training von Huggy** verwendet wurden, verstehen.
- **Deinen eigenen Huggy** darauf trainieren, den Stock zu holen.
- in der Lage sein, **mit deinem trainierten Huggy direkt in deinem Browser zu spielen**.




## Dieses Notebook stammt aus dem Deep Reinforcement Learning Kurs.
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

In diesem kostenlosen Kurs lernen Sie:

- 📖 Deep Reinforcement Learning in **Theorie und Praxis** studieren.
- 🧑‍💻 Lernen Sie, **berühmte Deep RL-Bibliotheken** wie Stable Baselines3, RL Baselines3 Zoo, CleanRL und Sample Factory 2.0 zu verwenden.
- 🤖 Trainieren Sie **Agenten in einzigartigen Umgebungen**.

Und mehr, siehe 📚 den Lehrplan 👉 https://simoninithomas.github.io/deep-rl-course

Vergessen Sie nicht, sich **<a href="http://eepurl.com/ic5ZUD">für den Kurs anzumelden</a>** (wir sammeln Ihre E-Mail, um Ihnen **die Links zu senden, wenn die einzelnen Einheiten veröffentlicht werden, und Sie über die Herausforderungen und Aktualisierungen zu informieren).**


Der beste Weg, um in Kontakt zu bleiben, ist, unserem Discord-Server beizutreten, um sich mit der Community und mit uns auszutauschen 👉🏻 https://discord.gg/ydHrjt3WP5

## Voraussetzungen 🏗️

Bevor Sie sich mit dem Notebook beschäftigen, müssen Sie:

🔲 📚 **ein Verständnis für die Grundlagen des Verstärkungslernens** (MC, TD, Belohnungshypothese...) entwickeln, indem Sie Einheit 1 bearbeiten

🔲 📚 **Lesen Sie die Einführung zu Huggy**, indem Sie die Bonuseinheit 1 bearbeiten

## Die GPU einstellen 💪.
- Um **das Training des Agenten zu beschleunigen, werden wir einen Grafikprozessor** verwenden. Gehen Sie dazu auf "Runtime > Change Runtime type".

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Schritt 1">

- Hardware-Beschleuniger > GPU".

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Schritt 2">

## Klonen Sie das Repository und installieren Sie die Abhängigkeiten 🔽.

- Wir müssen das Repository klonen, das **ML-Agents.** enthält.

In [ ]:
%%capture
# Clone the repository (can take 3min)
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents

In [ ]:
%%capture
# Go inside the repository and install the package (can take 3min)
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

## Laden Sie die Umgebungs-Zip-Datei herunter und verschieben Sie sie nach `./trained-envs-executables/linux/`.

- Unsere ausführbare Umgebungsdatei befindet sich in einer Zip-Datei.
- Wir müssen sie herunterladen und sie in `./trained-envs-executables/linux/` ablegen.

In [ ]:
!mkdir ./trained-envs-executables
!mkdir ./trained-envs-executables/linux

Wir haben die Datei Huggy.zip von https://github.com/huggingface/Huggy mit `wget` heruntergeladen

In [ ]:
!wget "https://github.com/huggingface/Huggy/raw/main/Huggy.zip" -O ./trained-envs-executables/linux/Huggy.zip

In [ ]:
%%capture
!unzip -d ./trained-envs-executables/linux/ ./trained-envs-executables/linux/Huggy.zip

Stellen Sie sicher, dass Ihre Datei zugänglich ist

In [ ]:
!chmod -R 755 ./trained-envs-executables/linux/Huggy

## Rekapitulieren wir, wie diese Umgebung funktioniert

### Der Zustandsraum: was Huggy "wahrnimmt".

Huggy "sieht" seine Umgebung nicht. Stattdessen liefern wir ihm Informationen über die Umgebung:

- Die Position des Ziels (Stock)
- Die relative Position zwischen ihm und dem Ziel
- Die Ausrichtung seiner Beine.

Mit all diesen Informationen kann Huggy **entscheiden, welche Aktion er als Nächstes durchführen muss, um sein Ziel zu erreichen**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.jpg" alt="Huggy" width="100%">


### Der Aktionsraum: welche Bewegungen Huggy machen kann
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-action.jpg" alt="Huggy Aktion" width="100%">

**Gelenkmotoren treiben Huggys Beine an**. Das heißt, um das Ziel zu erreichen, muss Huggy **lernen, die Gelenkmotoren jedes seiner Beine richtig zu drehen, damit er sich bewegen kann**.

### Die Belohnungsfunktion

Die Belohnungsfunktion ist so konzipiert, dass **Huggy sein Ziel erreicht**: den Stock zu holen.

Denken Sie daran, dass eine der Grundlagen des Verstärkungslernens die *Belohnungshypothese* ist: ein Ziel kann als **Maximierung der erwarteten kumulativen Belohnung** beschrieben werden.

Hier ist unser Ziel, dass Huggy **zum Stock geht, ohne sich zu sehr zu drehen**. Daher muss unsere Belohnungsfunktion dieses Ziel umsetzen.

Unsere Belohnungsfunktion:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/reward.jpg" alt="Huggy Belohnungsfunktion" width="100%">

- *Orientierungsbonus*: wir **belohnen ihn, wenn er sich dem Ziel nähert**.
- *Zeitstrafe*: eine feste Zeitstrafe bei jeder Aktion, um ihn zu **zwingen, so schnell wie möglich zum Stock zu kommen**.
- Rotationsstrafe*: Wir bestrafen Huggy, wenn er sich **zu sehr dreht und zu schnell dreht**.
- *Belohnung für das Erreichen des Ziels*: Wir belohnen Huggy für das **Erreichen des Ziels**.

## Erstellen Sie die Huggy-Konfigurationsdatei

- In ML-Agents definieren Sie die **Trainings-Hyperparameter in config.yaml-Dateien.**

- Im Rahmen dieses Notizbuchs werden wir die Hyperparameter nicht ändern, aber wenn Sie es als Experiment versuchen wollen, sollten Sie auch versuchen, einige andere Hyperparameter zu ändern. Unity bietet eine sehr [gute Dokumentation, die jeden von ihnen hier erklärt] (https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Training-Configuration-File.md).

- Aber wir müssen eine Konfigurationsdatei für Huggy erstellen.

  - Klicke dazu auf das Ordner-Logo auf der linken Seite deines Bildschirms.

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/create_file.png" alt="Datei erstellen" width="10%">

  - Gehen Sie zu `/content/ml-agents/config/ppo`
  - Klicken Sie mit der rechten Maustaste und erstellen Sie eine neue Datei namens `Huggy.yaml`.

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/create-huggy.png" alt="Erstelle huggy.yaml" width="20%">

- Kopieren Sie den folgenden Inhalt und fügen Sie ihn ein 🔽.

In [ ]:
behaviors:
  Huggy:
    trainer_type: ppo
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: true
      hidden_units: 512
      num_layers: 3
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.995
        strength: 1.0
    checkpoint_interval: 200000
    keep_checkpoints: 15
    max_steps: 2e6
    time_horizon: 1000
    summary_freq: 50000

- Vergessen Sie nicht, die Datei zu speichern!

- Falls Sie die Hyperparameter** ändern möchten, können Sie in Google Colab notebook hier klicken, um die config.yaml zu öffnen: `/content/ml-agents/config/ppo/Huggy.yaml`

- Zum Beispiel **wenn Sie mehr Modelle während des Trainings speichern wollen** (im Moment speichern wir alle 200.000 Trainingszeitschritte). Sie müssen Folgendes ändern:
  - `checkpoint_interval`: Die Anzahl der Trainingszeitschritte, die zwischen den einzelnen Checkpoints gesammelt werden.
  - keep_checkpoints": Die maximale Anzahl der zu speichernden Modellprüfpunkte.

=> Behalten Sie im Hinterkopf, dass **eine Verringerung des `checkpoint_interval` bedeutet, dass mehr Modelle zum Hub hochgeladen werden müssen und somit eine längere Hochladezeit**
Wir sind nun bereit, unseren Agenten zu trainieren 🔥.

## Ausbildung unseres Agenten

Um unseren Agenten zu trainieren, müssen wir nur **mlagents-learn starten und die ausführbare Datei auswählen, die die Umgebung enthält**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mllearn.png" alt="ml learn function" width="100%">

Mit ML Agents führen wir ein Trainingsskript aus. Wir definieren vier Parameter:

1. `mlagents-learn <config>`: der Pfad, in dem sich die Konfigurationsdatei für die Hyperparameter befindet.
2. `--env`: wo sich die ausführbare Umgebung befindet.
3. Run-id": Der Name, den Sie Ihrer Trainingslauf-ID geben wollen.
4. `--no-graphics`: um die Visualisierung während des Trainings nicht zu starten.

Trainieren Sie das Modell und verwenden Sie das `--resume` Flag, um das Training im Falle einer Unterbrechung fortzusetzen.

> Es wird beim ersten Mal fehlschlagen, wenn Sie `--resume` verwenden, versuchen Sie den Block erneut auszuführen, um den Fehler zu umgehen.



Das Training dauert zwischen 30 und 45 Minuten, je nach Computer (vergessen Sie nicht, eine GPU** einzurichten), gehen Sie auf ☕️you und verdienen Sie es 🤗.

In [ ]:
!mlagents-learn ./config/ppo/Huggy.yaml --env=./trained-envs-executables/linux/Huggy/Huggy --run-id="Huggy2" --no-graphics

## Schieben Sie den Agenten zum 🤗 Hub

- Nun, da wir unseren Agenten trainiert haben, sind wir **bereit, ihn zum Hub zu schieben, um mit Huggy im Browser spielen zu können🔥.**

Um Ihr Modell mit der Gemeinschaft teilen zu können, sind drei weitere Schritte erforderlich:

1️⃣ (Falls noch nicht geschehen) Erstellen Sie ein Konto für HF ➡ https://huggingface.co/join

2️⃣ Melde dich an und speichere dann dein Authentifizierungs-Token von der Hugging Face Website.
- Erstellen Sie ein neues Token (https://huggingface.co/settings/tokens) **mit Schreibrolle**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="HF-Token erstellen">

- Kopieren Sie das Token
- Führen Sie die Zelle unten aus und fügen Sie das Token ein

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Wenn Sie kein Google Colab oder ein Jupyter Notebook verwenden möchten, müssen Sie stattdessen diesen Befehl verwenden: `huggingface-cli login`

Dann müssen wir einfach `mlagents-push-to-hf` ausführen.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mlpush.png" alt="ml learn function" width="100%">

Und wir definieren 4 Parameter:

1. Lauf-ID": der Name des Trainingslaufs.
2. `--local-dir`: wo der Agent gespeichert wurde, es ist results/<run_id name>, also in meinem Fall results/First Training.
3. `--repo-id`: der Name des Hugging Face Repos, das du erstellen oder aktualisieren willst. Es ist immer <Ihr Hugging-Face-Benutzername>/<Der Repo-Name>
Wenn das Repo nicht existiert, wird es automatisch erstellt**.
4. `--commit-message`: Da HF-Repos Git-Repos sind, müssen Sie eine Commit-Message definieren.

In [ ]:
!mlagents-push-to-hf --run-id="HuggyTraining" --local-dir="./results/Huggy" --repo-id="ThomasSimonini/ppo-Huggy" --commit-message="Huggy"

Andernfalls, wenn alles funktioniert hat, sollten Sie am Ende des Prozesses diese Seite haben (allerdings mit einer anderen URL 😆):



```
Ihr Modell wird in den Hub übertragen. Du kannst dein Modell hier sehen: https://huggingface.co/ThomasSimonini/ppo-Huggy
```

Das ist der Link zu Ihrem Modell-Repository. Das Repository enthält eine Modellkarte, die erklärt, wie man das Modell benutzt, Ihre Tensorboard-Logs und Ihre Konfigurationsdatei. **Das Tolle ist, dass es ein Git-Repository ist, was bedeutet, dass Sie verschiedene Commits haben können, Ihr Repository mit einem neuen Push aktualisieren können, Pull Requests öffnen können, etc.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/modelcard.png" alt="ml learn function" width="100%">

Aber jetzt kommt das Beste: **Mit Huggy online spielen zu können 👀.**

## Spiel mit deinem Huggy 🐕

Dieser Schritt ist der einfachste:

- Öffne das Spiel Huggy in deinem Browser: https://huggingface.co/spaces/ThomasSimonini/Huggy

- Klicken Sie auf Play with my Huggy model

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/load-huggy.jpg" alt="load-huggy" width="100%">

1. In Schritt 1 wählen Sie Ihr Modell-Repository, das die Modell-ID ist (in meinem Fall ThomasSimonini/ppo-Huggy).

2. In Schritt 2 **wählen Sie das Modell aus, das Sie wiedergeben möchten**:
  - Ich habe mehrere Modelle, da wir alle 500000 Zeitschritte ein Modell gespeichert haben.
  - Da ich aber das aktuellste möchte, wähle ich "Huggy.onnx".

👉 Das Schöne ist, **dass man verschiedene Modellschritte ausprobieren kann, um die Verbesserung des Agenten zu sehen**.

Herzlichen Glückwunsch zum Abschluss dieser Bonuseinheit!

Jetzt kannst du dich hinsetzen und mit deinem Huggy 🐶 spielen. Und vergesst **nicht, die Liebe zu verbreiten, indem ihr Huggy mit euren Freunden teilt 🤗**. Und wenn du es in den sozialen Medien teilst, **bitte tagge uns @huggingface und mich @simoninithomas**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-cover.jpeg" alt="Huggy-Cover" width="100%">


## Keep Learning, Stay awesome 🤗